# Preprocessing

This workflow assumes the data is in the following format:
- 2 very large .btf (bigtiff) files for the volumetric data

This workflow then performs the following steps:
1. Convert each channel to .avi
2. Creates a DLC folder

In later steps, the full DLC training will be used for tracking and, finally, trace extraction


## Uses a config file to define all properties for the project

In [1]:
from DLC_for_WBFM.bin.configuration_definition import *
from DLC_for_WBFM.utils.video_and_data_conversion.video_conversion_utils import write_minimax_projection_from_btf
import os
%load_ext autoreload
%autoreload 2

## First, create the overall project folder

In [2]:
# Initialize the config files
task_name = 'test'
experimenter = 'Charlie'

config = DLCForWBFMConfig(task_name=task_name, experimenter=experimenter)

In [3]:
# Actually create the project folder

# Note: Windows-style filename
working_directory = r'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects'

config = create_project(config, working_directory)

Project "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_test-Charlie-2020-12-15" already exists!
Saving config to filename 'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_test-Charlie-2020-12-15\config.pickle''


## Second, find the raw bigtiff files

In [5]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

### If you already converted to .avi, write them here

In [6]:
red_avi_fname = None
green_avi_fname = None

In [7]:
# Create the first sub-object
# This remembers the location of the raw data
datafiles = DLCForWBFMDatafiles(btf_fname_red, btf_fname_green, red_avi_fname, green_avi_fname)

config.datafiles = datafiles

In [8]:
print(config)

                Field values:
                task_name: test 
                experimenter: Charlie 
                config_filename: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_test-Charlie-2020-12-15\config.pickle
                Which subclasses are initialized?
                datafiles: True
                preprocessing: False
                tracking: False
                traces: False



## Third, do preprocessing

i.e. Convert .btf to .avi for DeepLabCut

In [14]:
# Set up options for the .avi video
opt = {'start_volume':           100, 
       'num_frames':             10, 
       'num_total_slices':       33, 
       'num_crop_slices':        11, 
       'center_slice':           12, 
       'alpha':                  0.15,
       'fps':                    5.0,
       'red_and_green_mirrored': True}
preprocessing = DLCForWBFMPreprocessing(**opt)

config.preprocessing = preprocessing

In [15]:
print(config)

                Field values:
                task_name: test 
                experimenter: Charlie 
                config_filename: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_test-Charlie-2020-12-15\config.pickle
                Which subclasses are initialized?
                datafiles: True
                preprocessing: True
                tracking: False
                traces: False



In [21]:
# Now make the .avi video, if not already made
# Note: if the .avi was made elsewhere, check that the options above are correct!
write_minimax_projection_from_btf(config)

Converting volume index 100 to frame index 3300
Taking a max of 11 slices, starting at 7
Page 3307/330; a portion of slice 0/10 to tmp array index 0
Page 3308/330; a portion of slice 0/10 to tmp array index 1
Page 3309/330; a portion of slice 0/10 to tmp array index 2
Page 3310/330; a portion of slice 0/10 to tmp array index 3
Page 3311/330; a portion of slice 0/10 to tmp array index 4
Page 3312/330; a portion of slice 0/10 to tmp array index 5
Page 3313/330; a portion of slice 0/10 to tmp array index 6
Page 3314/330; a portion of slice 0/10 to tmp array index 7
Page 3315/330; a portion of slice 0/10 to tmp array index 8
Page 3316/330; a portion of slice 0/10 to tmp array index 9
Page 3317/330; a portion of slice 0/10 to tmp array index 10
Page 3340/330; a portion of slice 1/10 to tmp array index 0
Page 3341/330; a portion of slice 1/10 to tmp array index 1
Page 3342/330; a portion of slice 1/10 to tmp array index 2
Page 3343/330; a portion of slice 1/10 to tmp array index 3
Page 3344/

Page 3373/330; a portion of slice 2/10 to tmp array index 0
Page 3374/330; a portion of slice 2/10 to tmp array index 1
Page 3375/330; a portion of slice 2/10 to tmp array index 2
Page 3376/330; a portion of slice 2/10 to tmp array index 3
Page 3377/330; a portion of slice 2/10 to tmp array index 4
Page 3378/330; a portion of slice 2/10 to tmp array index 5
Page 3379/330; a portion of slice 2/10 to tmp array index 6
Page 3380/330; a portion of slice 2/10 to tmp array index 7
Page 3381/330; a portion of slice 2/10 to tmp array index 8
Page 3382/330; a portion of slice 2/10 to tmp array index 9
Page 3383/330; a portion of slice 2/10 to tmp array index 10
Page 3406/330; a portion of slice 3/10 to tmp array index 0
Page 3407/330; a portion of slice 3/10 to tmp array index 1
Page 3408/330; a portion of slice 3/10 to tmp array index 2
Page 3409/330; a portion of slice 3/10 to tmp array index 3
Page 3410/330; a portion of slice 3/10 to tmp array index 4
Page 3411/330; a portion of slice 3/10 

# Now, Make a DLC project folder and prep for annotations

## Create a project and copy the mcherry from above

In [22]:
import deeplabcut


ModuleNotFoundError: No module named 'deeplabcut'

In [6]:
# mcherry video, which will be tracked
if os.name == 'nt':
    video_foldername = 'Y:\\shared_projects\\wbfm\\dat\\short_wbfm_z_stabilized'
else:
    video_foldername = '/users/charles.fieseler/shared_projects/wbfm/dat/short_wbfm_z_stabilized/'
    '202009282020-09-28_17-30-28_worm4_chemotaxis-channel-0-pco_camera1_bigtiff_mcherry_frames1000_slice17_22'
fname = f'202009282020-09-28_17-30-28_worm4_chemotaxis-channel-0-pco_camera1_bigtiff_mcherry_frames{num_frames}_slice{which_slices[0]}_{which_slices[-1]}.avi'
video_fname = os.path.join(video_foldername, fname)

In [8]:
if os.name == 'nt':
    project_foldername = 'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\DLC_Projects'
else:
    raise Exception
# fname = '202009282020-09-28_17-30-28_worm4_chemotaxis-channel-1-pco_camera2bigtiff.btf'
# input_fname = os.path.join(input_foldername, fname)
#     project_foldername = '/groups/zimmer/Ulises/wbfm/20200924and28/202009282020-09-28_17-30-28_worm4_chemotaxis-channel-1-pco_camera2/'

task = 'wbfm_3d'
experimenter = 'Charlie'


In [11]:
path_config_file = deeplabcut.create_new_project(task,experimenter,videos=[video_fname],copy_videos=True,working_directory=project_foldername) 

Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d_annotation_test-Charlie-2020-10-21\videos"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d_annotation_test-Charlie-2020-10-21\labeled-data"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d_annotation_test-Charlie-2020-10-21\training-datasets"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d_annotation_test-Charlie-2020-10-21\dlc-models"
Copying the videos
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d_annotation_test-Charlie-2020-10-21\videos\202009282020-09-28_17-30-28_worm4_chemotaxis-channel-0-pco_camera1_bigtiff_mcherry_frames1000_slice17_22.avi
Generated "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d_annotation_test-Charlie-2020-10-21\config.yaml"

A new project with name wbfm_3d_annotation_

## Extract individual volumes to be annotated

In [9]:
import deeplabcut
from DLC_for_WBFM.utils.preprocessing.custom_frame_extraction import *

In [10]:
# Get a couple volumes to annotate
num_training_volumes = 30
which_volumes = list(range(start_volume, start_volume + num_training_volumes))

# Take the entire z stack
which_slice = None
nz = 33

In [11]:
project_folder = 'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\DLC_Projects\\wbfm_3d-Charlie-2020-10-21\\'
path_config_file = os.path.join(project_folder, 'config.yaml')

video_fname = os.path.join(project_folder, 'videos\\mcherry_frames1000_slice17_22.btf')

In [13]:
extract_volume_from_tiff_in_dlc_project(path_config_file, nz=nz, which_vol=which_volumes, which_slice=which_slice, video_fname=video_fname)

Config file read successfully.
Reading volume 100/29
Converted volume indices: 3300 to 3332 (not including last frame)
Saved volume to C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22\img100.tif
Reading volume 101/29
Converted volume indices: 3333 to 3365 (not including last frame)
Saved volume to C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22\img101.tif
Reading volume 102/29
Converted volume indices: 3366 to 3398 (not including last frame)
Saved volume to C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22\img102.tif
Reading volume 103/29
Converted volume indices: 3399 to 3431 (not including last frame)
Saved volume to C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Ch

### Synchronize the chosen volumes with training frames

For now, just pops up the GUI to do it manually

In [14]:
deeplabcut.extract_frames(path_config_file, mode='manual')

Quitting for now!


## Now, use ICY to annotate in 3d
__DONE BY HAND AND SAVED IN CORRECT FOLDER__

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Convert the generated .xml file to DLC format
from DLC_for_WBFM.utils.training_data.DLC_XML_conversion_utils import *
import deeplabcut

In [18]:
project_folder = 'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\DLC_Projects\\wbfm_3d-Charlie-2020-10-21\\'
path_config_file = os.path.join(project_folder, 'config.yaml')

video_fname = os.path.join(project_folder, 'videos\\mcherry_frames1000_slice17_22.btf')

In [20]:
df = icy_xml_to_dlc(path_config_file)

Found .xml annotations: Icy_annotations.xml
Found 1 group(s) of tracks
Relative image names:
['labeled-data\\mcherry_frames1000_slice17_22\\img0000.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0001.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0002.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0003.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0004.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0005.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0006.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0007.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0008.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0009.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0010.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0011.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0012.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0013.png', 'labeled-data\\mcherry_frames1000_slice17_22\\img0014.png', 'label

In [22]:
# Check the annotations
deeplabcut.check_labels(path_config_file)

Creating images with labels by Charlie.
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22_labeled  already exists!
They are stored in the following folder: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!
